In [93]:
import pandas as pd
import numpy as np
import re
import time
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split

In [94]:
def CareerYear(x):
    if pd.isna(x):
        return -1
    if len(x)==0:
        return -1
    if x.find('n/a') > -1:
        return -1
    elif x.find("10+")>-1:
        return 11
    elif x.find('< 1') > -1:
        return 0
    else:
        return int(re.sub("\D", "", x))


def DescExisting(x):
    x2 = str(x)
    if x2 == 'nan':
        return 'no desc'
    else:
        return 'desc'


def ConvertDateStr(x,format):
    if str(x) == 'nan':
        return datetime.datetime.fromtimestamp(time.mktime(time.strptime('9900-1','%Y-%m')))
    else:
        return datetime.datetime.fromtimestamp(time.mktime(time.strptime(x,format)))


def MonthGap(earlyDate, lateDate):
    if lateDate > earlyDate:
        gap = relativedelta(lateDate,earlyDate)
        yr = gap.years
        mth = gap.months
        return yr*12+mth
    else:
        return 0


def MakeupMissing(x):
    if np.isnan(x):
        return -1
    else:
        return x

In [27]:
allData=pd.read_csv('application.csv',encoding="ISO-8859-1")

In [28]:
allData['term'] = allData['term'].apply(lambda x: int(x.replace(' months','')))

In [30]:
# 处理标签：Fully Paid是正常用户；Charged Off是违约用户
allData['y'] = allData['loan_status'].map(lambda x: int(x == 'Charged Off'))

In [31]:
'''
由于存在不同的贷款期限（term），申请评分卡模型评估的违约概率必须要在统一的期限中，且不宜太长，所以选取term＝36months的行本
'''

allData1 = allData.loc[allData.term == 36]

In [32]:
allData1.shape

(29095, 26)

In [33]:
# 划分数据集合
trainData, testData = train_test_split(allData1,test_size=0.4,random_state=666)

In [39]:
trainData.columns

Index(['member_id', 'loan_amnt', 'term', 'loan_status', 'int_rate',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'total_acc',
       'pub_rec_bankruptcies', 'issue_d', 'earliest_cr_line', 'y'],
      dtype='object')

In [40]:
list11=['int_rate','emp_length','desc','issue_d','earliest_cr_line','mths_since_last_delinq','mths_since_last_record','pub_rec_bankruptcies']

# trainData.loc[:,['int_rate','emp_length']]
trainData.loc[:,list11]

,int_rate,emp_length,desc,issue_d,earliest_cr_line,mths_since_last_delinq,mths_since_last_record,pub_rec_bankruptcies
36814,9.63%,8 years,This will be used to pay off an existing vehi...,May-09,Jun-91,NaN,NaN,0.0
328,9.91%,10+ years,Borrower added on 12/11/11 > This loan is to...,Dec-11,Mar-98,NaN,NaN,0.0
17761,16.02%,< 1 year,Borrower added on 03/24/11 > I am in love an...,Mar-11,Apr-07,NaN,64.0,0.0
10056,11.49%,3 years,I plan to use the loan funds to pay off my cre...,Aug-11,Feb-93,31.0,NaN,0.0
3965,6.03%,2 years,NaN,Nov-11,Dec-97,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
4241,6.62%,10+ years,Borrower added on 10/28/11 > pay off my cred...,Nov-11,Jan-99,NaN,NaN,0.0
37216,13.47%,2 years,Id like to pay off my credit card debt complet...,Apr-09,Feb-05,NaN,NaN,0.0
11409,5.42%,7 years,Borrower added on 07/14/11 > FOR 1995 WELLCR...,Jul-11,Mar-97,NaN,NaN,0.0
15217,8.49%,10+ years,NaN,May-11,Jan-98,15.0,NaN,0.0


In [95]:
# 将带％的百分比变为浮点数
trainData['int_rate_clean'] = trainData['int_rate'].map(lambda x: float(x.replace('%',''))/100)
# 将工作年限进行转化，否则影响排序
trainData['emp_length_clean'] = trainData['emp_length'].map(CareerYear)
# 将desc的缺失作为一种状态，非缺失作为另一种状态
trainData['desc_clean'] = trainData['desc'].map(DescExisting)

# 处理日期。earliest_cr_line的格式不统一，需要统一格式且转换成python的日期
trainData['app_date_clean'] = trainData['issue_d'].map(lambda x: ConvertDateStr(x,'%b-%y'))
trainData['earliest_cr_line_clean'] = trainData['earliest_cr_line'].map(lambda x: ConvertDateStr(x,'%b-%y'))

# 处理mths_since_last_delinq。注意原始值中有0，所以用－1代替缺失
trainData['mths_since_last_delinq_clean'] = trainData['mths_since_last_delinq'].map(lambda x:MakeupMissing(x))

trainData['mths_since_last_record_clean'] = trainData['mths_since_last_record'].map(lambda x:MakeupMissing(x))

trainData['pub_rec_bankruptcies_clean'] = trainData['pub_rec_bankruptcies'].map(lambda x:MakeupMissing(x))

/Users/gengbh/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/gengbh/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/gengbh/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [100]:
'''
第二步：变量衍生
'''
# 考虑申请额度与收入的占比
trainData['limit_income'] = trainData.apply(lambda x: x.loan_amnt / x.annual_inc, axis = 1)

# 考虑earliest_cr_line到申请日期的跨度，以月份记
trainData['earliest_cr_to_app'] = trainData.apply(lambda x: MonthGap(x.earliest_cr_line_clean,x.app_date_clean), axis = 1)

/Users/gengbh/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/gengbh/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [101]:
'''
第三步：分箱，采用ChiMerge,要求分箱完之后：
（1）不超过5箱
（2）Bad Rate单调
（3）每箱同时包含好坏样本
（4）特殊值如－1，单独成一箱

连续型变量可直接分箱
类别型变量：
（a）当取值较多时，先用bad rate编码，再用连续型分箱的方式进行分箱
（b）当取值较少时：
    （b1）如果每种类别同时包含好坏样本，无需分箱
    （b2）如果有类别只包含好坏样本的一种，需要合并
'''
num_features = ['int_rate_clean','emp_length_clean','annual_inc', 'dti', 'delinq_2yrs', 'earliest_cr_to_app','inq_last_6mths', \
                'mths_since_last_record_clean', 'mths_since_last_delinq_clean','open_acc','pub_rec','total_acc']

cat_features = ['home_ownership', 'verification_status','desc_clean', 'purpose', 'zip_code','addr_state','pub_rec_bankruptcies_clean']


In [104]:
more_value_features = []
less_value_features = []
# 第一步，检查类别型变量中，哪些变量取值超过5
for var in cat_features:
    valueCounts = len(set(trainData[var]))
    # print valueCounts
    if valueCounts > 5:
        more_value_features.append(var)
    else:
        less_value_features.append(var)


In [106]:
less_value_features

['home_ownership',
 'verification_status',
 'desc_clean',
 'pub_rec_bankruptcies_clean']

In [119]:
def BinBadRate(df, col, target, grantRateIndicator=0):
    '''
    :param df: 需要计算好坏比率的数据集
    :param col: 需要计算好坏比率的特征
    :param target: 好坏标签
    :param grantRateIndicator: 1返回总体的坏样本率，0不返回
    :return: 每箱的坏样本率，以及总体的坏样本率（当grantRateIndicator＝＝1时）
    '''
    total = df.groupby([col])[target].count()
    total = pd.DataFrame({'total': total})
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    regroup['bad_rate'] = regroup.apply(lambda x: x.bad * 1.0 / x.total, axis=1)
    dicts = dict(zip(regroup[col],regroup['bad_rate']))
    if grantRateIndicator==0:
        return (dicts, regroup)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    overallRate = B * 1.0 / N
    return (dicts, regroup, overallRate)
 ### If we find any categories with 0 bad, then we combine these categories with that having smallest non-zero bad rate
def MergeBad0(df,col,target):
    '''
     :param df: dataframe containing feature and target
     :param col: the feature that needs to be calculated the WOE and iv, usually categorical type
     :param target: good/bad indicator
     :return: WOE and IV in a dictionary
     '''
    regroup = BinBadRate(df, col, target)[1]
    regroup = regroup.sort_values(by  = 'bad_rate')
    col_regroup = [[i] for i in regroup[col]]
    for i in range(regroup.shape[0]-1):
        col_regroup[i+1] = col_regroup[i] + col_regroup[i+1]
        col_regroup.pop(i)
        if regroup['bad_rate'][i+1] > 0:
            break
    newGroup = {}
    for i in range(len(col_regroup)):
        for g2 in col_regroup[i]:
            newGroup[g2] = 'Bin '+str(i)
    return newGroup   


def BadRateEncoding(df, col, target):
    '''
    :param df: dataframe containing feature and target
    :param col: the feature that needs to be encoded with bad rate, usually categorical type
    :param target: good/bad indicator
    :return: the assigned bad rate to encode the categorical feature
    '''
    regroup = BinBadRate(df, col, target, grantRateIndicator=0)[1]
    br_dict = regroup[[col,'bad_rate']].set_index([col]).to_dict(orient='index')
    for k, v in br_dict.items():
        br_dict[k] = v['bad_rate']
    badRateEnconding = df[col].map(lambda x: br_dict[x])
    return {'encoding':badRateEnconding, 'bad_rate':br_dict}

In [114]:
# （i）当取值<5时：如果每种类别同时包含好坏样本，无需分箱；如果有类别只包含好坏样本的一种，需要合并
merge_bin = {}
for col in less_value_features:
    binBadRate = BinBadRate(trainData, col, 'y')[0]
    if min(binBadRate.values()) == 0 or max(binBadRate.values()) == 1:
        print('{} need to be combined'.format(col))
        combine_bin = MergeBad0(trainData, col, 'y')
        merge_bin[col] = combine_bin



home_ownership need to be combined
pub_rec_bankruptcies_clean need to be combined


In [117]:
merge_bin.get('home_ownership')

{'NONE': 'Bin 0',
 'MORTGAGE': 'Bin 0',
 'OWN': 'Bin 1',
 'RENT': 'Bin 1',
 'OTHER': 'Bin 2'}

In [121]:

# （ii）当取值>5时：用bad rate进行编码，放入连续型变量里
br_encoding_dict = {}
for col in more_value_features:
    br_encoding = BadRateEncoding(trainData, col, target)
    trainData[col+'_br_encoding'] = br_encoding['encoding']
    br_encoding_dict[col] = br_encoding['bad_rate']
    num_features.append(col+'_br_encoding')


NameError: name 'target' is not defined

In [ ]:

# （iii）对连续型变量进行分箱，包括（ii）中的变量
for col in num_features:
    print "{} is in processing".format(col)
    if -1 not in set(trainData[col]):
        max_interval = 5
        cutOff = ChiMerge(trainData, col, target, max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        trainData[col+'_Bin'] = trainData[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(trainData, col+'_Bin', 'y')
        while(not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData, col, target, max_interval=max_interval, special_attribute=[],
                                          minBinPcnt=0)
            trainData[col + '_Bin'] = trainData[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[]))
            if max_interval == 2:
                # 当分箱数为2时，必然单调
                break
            monotone = BadRateMonotone(trainData, col + '_Bin', 'y')
    else:
        max_interval = 5
        cutOff = ChiMerge(trainData, col, target, max_interval=max_interval, special_attribute=[-1],
                                      minBinPcnt=0)
        trainData[col + '_Bin'] = trainData[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1]))
        monotone = BadRateMonotone(trainData, col + '_Bin', 'y')
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(trainData, col, target, max_interval=max_interval, special_attribute=[-1],
                                          minBinPcnt=0)
            trainData[col + '_Bin'] = trainData[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1]))
            if max_interval == 2:
                # 当分箱数为2时，必然单调
                break
            monotone = BadRateMonotone(trainData, col + '_Bin', 'y')